In [1]:
import requests
import yaml
import time
from tqdm import tqdm
from datetime import datetime
import steamreviews
from time import sleep
import os
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute

In [2]:
resp = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2")
app_list = resp.json()
apps = app_list['applist']['apps']

with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)


In [3]:
len(apps)

131986

In [4]:
def review_insert(reviews, appid):
    need_data = ['votes_up', 'votes_funny', 'steam_purchase', 'received_for_free', 'written_during_early_access']
    need_data_dic ={'steam_appid': appid}
        
    for _, review in reviews[0]['reviews'].items():
        author = review['author']
        steamid = author['steamid']
        sql = "SELECT steam_appid FROM review WHERE steam_appid=%s AND steamid=%s" % (appid, steamid)
        if db_function.execute_select(sql):
            return
        
        for k, v in author.items():
            
            if k == 'last_played':
                v = datetime.fromtimestamp(v).strftime("%Y-%m-%d %H:%M:%S")
            need_data_dic[k] = v
        
        for col in need_data:
            need_data_dic[col] = review[col]
        
        placeholders = ', '.join(['%s'] * len(need_data_dic))
        columns = ', '.join(need_data_dic.keys())
        
        sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s )" % ('review', columns, placeholders)
        db_function.execute_insert(sql, list(need_data_dic.values()))


In [7]:
start_time = time.time()
try_cnt = 0

db_function = None
try:
    db_function = db_execute(db_auth['local_db']["host"], 
                         db_auth['local_db']["port"],
                         db_auth['local_db']["user"],
                         db_auth['local_db']["db"], 
                         db_auth['local_db']["passwd"])
    
    for app in tqdm(apps):
        appid=app['appid']
        
        sql = "SELECT id FROM game_info WHERE steam_appid=%s" % appid
        if not db_function.execute_select(sql):
            continue
        try_cnt +=1
        current_time = time.time()
        elapsed_time = current_time - start_time

        if try_cnt > 200 and elapsed_time < 300:
            sleep(60)
            try_cnt = 0
            start_time = time.time()
            

        
        request_params = dict()
        request_params['language'] = 'korean'

        reviews = steamreviews.download_reviews_for_app_id(appid,chosen_request_params=request_params)
        print(reviews)
        if reviews[0]['reviews']:
            review_insert(reviews, appid)
        sleep(5)
except:
    raise
finally:
    db_function.curs.close()
    db_function.conn.close()
    

  0%|                                                                                                                                                                                                                                                    | 0/131986 [00:00<?, ?it/s]

({'reviews': {}, 'query_summary': {'total_reviews': -1}, 'cursors': {'*': 'Thu Dec  9 15:18:20 2021'}}, 1)


  0%|                                                                                                                                                                                                                                        | 1/131986 [00:05<192:15:26,  5.24s/it]

({'reviews': {}, 'query_summary': {'total_reviews': -1}, 'cursors': {'*': 'Thu Dec  9 15:18:25 2021'}}, 1)


  0%|                                                                                                                                                                                                                                        | 2/131986 [00:10<191:57:56,  5.24s/it]

...

({'reviews': {}, 'query_summary': {'total_reviews': -1}, 'cursors': {'*': 'Thu Dec  9 15:21:50 2021'}}, 1)


  0%|                                                                                                                                                                                                                                       | 41/131986 [03:35<192:50:01,  5.26s/it]

({'reviews': {}, 'query_summary': {'total_reviews': -1}, 'cursors': {'*': 'Thu Dec  9 15:21:55 2021'}}, 1)


  0%|                                                                                                                                                                                                                                       | 41/131986 [03:40<197:33:10,  5.39s/it]


KeyboardInterrupt: 